In [4]:
import  numpy as np  
import pandas as pd
from sklearn.preprocessing  import OneHotEncoder

In [5]:
df=pd.read_csv("/kaggle/input/textiledefectdetection/train64.csv")
df=df.drop(columns=["split"])

In [8]:
df.isnull().sum()

index               0
angle               0
indication_type     0
indication_value    0
dtype: int64

In [9]:
one_hot=OneHotEncoder(sparse_output=False)
one=one_hot.fit_transform(df[["indication_type"]])
one=pd.DataFrame(one,columns=one_hot.get_feature_names_out())

In [10]:
one

,indication_type_color,indication_type_cut,indication_type_good,indication_type_hole,indication_type_metal_contamination,indication_type_thread
0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...
47995,0.0,0.0,0.0,0.0,0.0,1.0
47996,0.0,0.0,0.0,0.0,0.0,1.0
47997,0.0,0.0,0.0,0.0,0.0,1.0
47998,0.0,0.0,0.0,0.0,0.0,1.0


In [11]:
new_df=pd.concat([df,one],axis=1).drop(columns=["indication_type"])

In [12]:
new_df

,index,angle,indication_value,indication_type_color,indication_type_cut,indication_type_good,indication_type_hole,indication_type_metal_contamination,indication_type_thread
0,48000,0,0,0.0,0.0,1.0,0.0,0.0,0.0
1,48001,80,0,0.0,0.0,1.0,0.0,0.0,0.0
2,48002,100,0,0.0,0.0,1.0,0.0,0.0,0.0
3,48003,100,0,0.0,0.0,1.0,0.0,0.0,0.0
4,48004,0,0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
47995,95995,20,4,0.0,0.0,0.0,0.0,0.0,1.0
47996,95996,20,4,0.0,0.0,0.0,0.0,0.0,1.0
47997,95997,140,4,0.0,0.0,0.0,0.0,0.0,1.0
47998,95998,120,4,0.0,0.0,0.0,0.0,0.0,1.0


# # Deep learning AutoEncoder

In [19]:
from keras.layers import Dense,Input,Dropout,BatchNormalization,LeakyReLU
from keras.models import Model
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [14]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1)

In [26]:
inputs=Input(shape=(9,))
e=Dense(90,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
e=LeakyReLU(0.8)(e)
e=Dropout(0.5)(e)
e=Dense(60,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(e)
e=LeakyReLU(0.8)(e)

e=Dense(30,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(e)
e=LeakyReLU(0.8)(e)
e=Dropout(0.5)(e)

e=Dense(9,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(e)
e=LeakyReLU(0.8)(e)

d=Dense(9,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(e)
d=LeakyReLU(0.8)(d)
d=Dropout(0.5)(d)
d=Dense(30,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(d)
d=LeakyReLU(0.8)(d)

d=Dense(60,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(d)
d=LeakyReLU(0.8)(d)
d=Dropout(0.5)(d)

d=Dense(90,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(d)
d=LeakyReLU(0.8)(d)
outputs=Dense(9,activation='sigmoid')(d)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [27]:
model.fit(new_df,new_df,epochs=32,batch_size=32,validation_split=0.2,callbacks=[early_stopping])

Epoch 1/32
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.9711 - loss: nan - val_accuracy: 1.0000 - val_loss: nan
Epoch 2/32
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 1.0000 - loss: nan - val_accuracy: 1.0000 - val_loss: nan
Epoch 3/32
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 1.0000 - loss: nan - val_accuracy: 1.0000 - val_loss: nan
Epoch 4/32
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 1.0000 - loss: nan - val_accuracy: 1.0000 - val_loss: nan
Epoch 5/32
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 1.0000 - loss: nan - val_accuracy: 1.0000 - val_loss: nan
Epoch 5: early stopping


In [35]:
prediction = model.predict(new_df)
reconstruction_error = np.mean(np.square(new_df - prediction), axis=1)
threshold = np.mean(reconstruction_error) + 2 * np.std(reconstruction_error)
# //0 for anomalies, 1 for normal
labels = np.where(reconstruction_error > threshold, 0, 1)
anomaly_percentage = np.mean(labels==0) * 100
print(f"Percentage of anomalies detected: {anomaly_percentage:.2f}%")


1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
Percentage of anomalies detected: 0.00%


In [34]:
pd.DataFrame(labels)

,0
0,1
1,1
2,1
3,1
4,1
...,...
47995,1
47996,1
47997,1
47998,1
